# Time Series Forecasting with LSTM - Pytorch

- Since I intend to predict both the Sales Value and Product Name, given the customer name and city, we will develop a model using deep learning techniques, not limited to `LSTM` but I am experimenting that first.
- The steps included in this notebook are for experimenting before choosing the model to add to the live trainng pipeline for production.

I am Glad🤓💻

**Important Notes**

- Predicting the Sales Value is a Linear function, thus I will utilize `nn.Linear` class to create it's linear model
- Predicting the **Product Name** is a classification problem, thus we will have to assign all classes a probability as we predict them. (`Softmax` is a good fit)

In [1]:
import torch
import torchmetrics
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
import pandas as pd
import numpy as np
import os
import random
import time
import warnings
from pathlib import Path

warnings.filterwarnings("ignore")

In [2]:
# TODO: Load the train and validation data
data_path = (Path.cwd() / "../data/processed").resolve()
print(f"Loading data from {data_path}")
train_file = data_path / "train.csv"
val_file = data_path / "val.csv"

if not train_file.exists() or not val_file.exists():
	print("Data files not found.")
else:
	train_data = pd.read_csv(train_file)
	val_data = pd.read_csv(val_file)

Loading data from /home/dan/Desktop/time_series_analysis/data/processed


In [3]:
train_data.head()

,Sub-Category,Product Name,dayofweek_cos,Customer Name,State,City,month_cos,Sales
0,5,790,1.000000,380,40,271,-8.660254e-01,1.140088
1,12,1720,0.623490,645,3,415,-5.000000e-01,-0.129866
2,3,1589,0.623490,357,11,388,1.000000e+00,-0.394460
3,10,210,0.623490,700,28,346,-1.836970e-16,-0.362179
4,13,1135,-0.900969,686,20,115,-5.000000e-01,0.551033


In [4]:
train_data = train_data[["Sub-Category", "Customer Name", "State", "City", "dayofweek_cos", "month_cos","Product Name", "Sales"]]
train_data.head()

,Sub-Category,Customer Name,State,City,dayofweek_cos,month_cos,Product Name,Sales
0,5,380,40,271,1.000000,-8.660254e-01,790,1.140088
1,12,645,3,415,0.623490,-5.000000e-01,1720,-0.129866
2,3,357,11,388,0.623490,1.000000e+00,1589,-0.394460
3,10,700,28,346,0.623490,-1.836970e-16,210,-0.362179
4,13,686,20,115,-0.900969,-5.000000e-01,1135,0.551033


In [5]:
# TODO: Train the Linear model for predicting sales
class LinearModel(nn.Module):
    def __init__(self, input_dim: int, output_dim: int) -> None:
        super().__init__()
        self.linear = nn.Linear(input_dim, output_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.3)

    def forward(self, x):
        x = self.linear(x)
        x = self.relu(x)
        x = self.dropout(x)
        return x

# set random seed for reproducability
random.seed(42)
torch.manual_seed(42)
np.random.seed(42)

# TODO: Create the dataset class for training
class SalesDataset(Dataset):
    def __init__(self, data: pd.DataFrame) -> None:
        super().__init__()
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        features = torch.tensor(row[:-1].values, dtype=torch.float32)
        target = torch.tensor(row[-1], dtype=torch.float32)
        return features, target

# TODO: Create the dataset for training and validation
train_dataset = SalesDataset(train_data)
val_dataset = SalesDataset(val_data)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# TODO: Initialize the model, loss func and optimizer for a training loop of 5 epochs
input_dim = train_data.shape[1] - 1
output_dim = 1
model = LinearModel(input_dim, output_dim)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, weight_decay=0.001)

metrics = torchmetrics.MeanAbsoluteError()
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# TODO: Training the model
for epoch in range(num_epochs):
    for i, (features, target) in enumerate(train_loader):
        features = features.to(device)
        target = target.to(device)

        optimizer.zero_grad()
        outputs = model(features)

        loss = criterion(outputs, target.view(-1, 1))
        mae = metrics(outputs, target.view(-1, 1))

        loss.backward()
        optimizer.step()
        if (i + 1) % 100 == 0:
            print(f"Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {loss.item():.4f}, MAE: {mae.item():.4f}")

# TODO: Evaluate the model on the validation set
model.eval()
val_loss = 0.0
with torch.no_grad():
    for features, target in val_loader:
        features = features.to(device)
        target = target.to(device)

        outputs = model(features)
        loss = criterion(outputs, target.view(-1, 1))
        mae = metrics(outputs, target.view(-1, 1))
        val_loss += loss.item()
    val_loss /= len(val_loader)
print(f"Validation Loss: {val_loss:.4f}, MAE: {mae.item():.4f}")

Epoch [1/10], Step [100/221], Loss: 0.1998, MAE: 0.3557
Epoch [1/10], Step [200/221], Loss: 0.4188, MAE: 0.4362
Epoch [2/10], Step [100/221], Loss: 0.3201, MAE: 0.4245
Epoch [2/10], Step [200/221], Loss: 7.9663, MAE: 0.9135
Epoch [3/10], Step [100/221], Loss: 0.2347, MAE: 0.4124
Epoch [3/10], Step [200/221], Loss: 0.2218, MAE: 0.3979
Epoch [4/10], Step [100/221], Loss: 0.1468, MAE: 0.3488
Epoch [4/10], Step [200/221], Loss: 0.1432, MAE: 0.3113
Epoch [5/10], Step [100/221], Loss: 0.3265, MAE: 0.4721
Epoch [5/10], Step [200/221], Loss: 0.6026, MAE: 0.4619
Epoch [6/10], Step [100/221], Loss: 0.3336, MAE: 0.3768
Epoch [6/10], Step [200/221], Loss: 0.3877, MAE: 0.4355
Epoch [7/10], Step [100/221], Loss: 0.1199, MAE: 0.3201
Epoch [7/10], Step [200/221], Loss: 0.1595, MAE: 0.3322
Epoch [8/10], Step [100/221], Loss: 0.3086, MAE: 0.3622
Epoch [8/10], Step [200/221], Loss: 0.1290, MAE: 0.2937
Epoch [9/10], Step [100/221], Loss: 0.3312, MAE: 0.4174
Epoch [9/10], Step [200/221], Loss: 2.4078, MAE:

In [6]:
# class ProductPredictor(nn.Module):
#     def __init__(self, input_dim: int, output_dim: int):
#         super().__init__()
#         self.linear = nn.LSTM(input_dim, output_dim)
#         self.relu = nn.SoftMax()
#         self.dropout = nn.Dropout(p=0.3)

#     def forward(self, x):
#         x = self.linear(x)
#         x = self.relu(x)
#         x = self.dropout(x)
#         return x